# Example ***IBGE API*** data Goias
### Exemplo dados do ***IBGE API*** Goiás

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import folium

In [ ]:
#Acesso API Ibge malhas GO
url = 'http://servicodados.ibge.gov.br/api/v2/malhas/52/?resolucao=5'
headers = {'Accept': 'application/vnd.geo+json'}
r = requests.get(url, headers=headers)
#Resposta do API em JSON
j = r.json()
#Visualuar chaves JSON
print(j.keys())
print('############################################')

#Acesso API Ibge Localidades id e nome municipios GO
urlmun = requests.get('http://servicodados.ibge.gov.br/api/v1/localidades/estados/52/municipios')
mun = urlmun.json()
#Visualizar lista mun JSON
print(mun[1])

In [ ]:
#DataFrame apenas com id e nome
dfmun = pd.DataFrame(mun, columns=['id', 'nome'])

In [ ]:
#Restante dos dados de localidades
dfrestante = pd.DataFrame(mun, columns=['microrregiao'])
#Microrregião
dfmicro = pd.DataFrame.from_dict(dfrestante['microrregiao'].to_dict(), orient='index')
dfmicro.rename(columns={'id': 'idmicro', 'nome':'nomemicro'}, inplace=True)
#Mesorregiao
dfmeso = pd.DataFrame.from_dict(dfmicro['mesorregiao'].to_dict(), orient='index')
dfmeso.rename(columns={'id': 'idmeso','nome':'nomemeso'}, inplace=True)
#UF
dfuf = pd.DataFrame.from_dict(dfmeso['UF'].to_dict(), orient='index')
dfuf.rename(columns={'id': 'iduf','sigla':'siglauf', 'nome':'nomeuf'}, inplace=True)
#Regiao
dfregiao = pd.DataFrame.from_dict(dfuf['regiao'].to_dict(), orient='index')
dfregiao.rename(columns={'id': 'idregiao', 'sigla':'siglaregiao', 'nome':'nomeregiao'}, inplace=True)

In [ ]:
#Unir todos DataFrames
result = pd.concat([dfmun, dfrestante, dfmicro, dfmeso, dfuf, dfregiao], axis=1)
#result = pd.concat([dfmun, dfrestante, dfmicro, dfmeso, dfuf, dfregiao], axis=1, keys=['x1','x2', 'x3', 'x4', 'x5', 'x6'])
dframes = result.drop(columns=['microrregiao', 'mesorregiao', 'UF', 'regiao'])

In [ ]:
dframes.head(3)

In [ ]:
#Converter JSON informando keys em GeoDataFrame
goias = gpd.GeoDataFrame.from_features(j['features'])

In [ ]:
goias.head(3)

In [ ]:
goias.shape

In [ ]:
#mapa usando folium
geojson=goias.to_json()
m = folium.Map(location=[-16, -49], tiles='Cartodb Positron', zoom_start=6)

folium.GeoJson(geojson).add_to(m)

#m.save(os.path.join('results', 'GeoJSON_and_choropleth_3.html'))
m